<H3>Trabalho Prático 1 - Algoritmos 2 2024.1 UFMG</H3>
Membros:

* Gabriel Arcanjo Campelo Fadoul - 2021031645
* Mateus Gonçalves Moreira - 2022035610
* Victor Augusto Hon Fonseca - 2022035814

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from TriangProntaAlterada import *

<H4> Carga das instâncias de testes

In [2]:
with open("instances\\agp2009a-orthorand\\agp2009a-orthorand\\random-20-1.pol") as f:
    line = f.readline()
    file_values = line.split()
total_vertices = int(file_values[0])

G = Grafo()

V = []
for i in range(1,len(file_values),2):
    X, Y = file_values[i], file_values[i+1]
    
    a, b = X.split("/")
    coord_x = float(a) / float(b)
    
    a, b = Y.split("/")
    coord_y = float(a) / float(b)
    
    V.append(Vertice(coord_x, coord_y))
    G.adicionar_vertice(Vertice(coord_x, coord_y))

for i in range(len(V)-1):
    G.adicionar_aresta(Aresta(G.vertices[i],G.vertices[i+1]))
G.adicionar_aresta(Aresta(G.vertices[len(V)-1],G.vertices[0]))

<H4>Funções Primitivas de Geometria Computacional

<H4>Funções de Triangulação

<H4>Animações

In [5]:
verticesAnimacao = list(G.vertices)
verticesAnimacao.append(G.vertices[0])
v2 = list(map(lambda P: (P.x,P.y), verticesAnimacao))
polygon_stages = []
polygon_stages.append(v2)
years = []
(G, triangulos, polygon_stages2, years2) = triangulacao(G, polygon_stages, years)

In [6]:
def color_triangle(graph, triangulos, i):
    colors = ['red', 'yellow', 'green']
    
    if i == 0:
        p1,p2,p3 = triangulos[i]
        graph.nodes[p1]['color'] = colors[0]
        graph.nodes[p2]['color'] = colors[1]
        graph.nodes[p3]['color'] = colors[2]
        actual_stage = (list(nx.get_node_attributes(graph,'color').values()))
        return graph, actual_stage

    ja_coloridos = dict()
    point = 0
    for p in triangulos[i]:
        if graph.nodes[p]['color'] != None:
            ja_coloridos[p] = graph.nodes[p]['color']
        else:
            point = p    
            
    to_color = list((set(colors) - (set(ja_coloridos.values()) & set(colors))))[0]
    graph.nodes[point]['color'] = to_color
    actual_stage = (list(nx.get_node_attributes(graph,'color').values()))
    return graph, actual_stage



In [7]:
colored_Graph = G.graph.copy()

dual = nx.Graph()
for i in range(len(triangulos)):
    tri_v = '-'.join(np.array(triangulos[i],dtype=str))
    dual.add_node(i,tri_v = tri_v)

for i in range(len(triangulos)):
    for j in range(len(triangulos)):
        if(len(set(triangulos[i]+triangulos[j]))) == 4:
            dual.add_edge(i,j)

dfs_edges = list(nx.dfs_edges(dual))
colored_Graph, act_s = color_triangle(colored_Graph,triangulos,0)
stages = [act_s]
for i,j in dfs_edges:
    colored_Graph, act_s = color_triangle(colored_Graph,triangulos,j)
    stages.append(act_s)

In [8]:
colors  = list(nx.get_node_attributes(colored_Graph,'color').values())

In [9]:
x_v, y_v = zip(*v)
x_v = list(x_v)
y_v = list(y_v)
xMin = min(x_v) - 3
yMin = min(y_v) - 3
xMax = max(x_v) + 3
yMax = max(y_v) + 3

In [10]:
# Define as coordenadas do polígono em diferentes estágios
polygon_stages = polygon_stages2

# Definir os anos como estágios do polígono para a animação
years = years2

# Criação da estrutura da figura
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# Configuração do layout
fig_dict["layout"]["width"] = 900 
fig_dict["layout"]["height"] = 700
fig_dict["layout"]["xaxis"] = {"range": [xMin, xMax], "title": "X"}
fig_dict["layout"]["yaxis"] = {"range": [yMin, yMax], "title": "Y"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Stage:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# Configuração inicial do polígono
initial_polygon = polygon_stages[0]
x, y = zip(*initial_polygon)
colors = ['blue'] * (len(x) - 1) + ['red']  # Último ponto em vermelho
data_dict = {
    "x": x,
    "y": y,
    "mode": "lines+markers",
    "marker": {"color": colors},
    "name": "Polygon",
    "line": {"shape": "linear"}
}
fig_dict["data"].append(data_dict)

# Criação dos frames para cada estágio do polígono
for i, year in enumerate(years):
    frame = {"data": [], "name": str(year)}
    polygon = polygon_stages[i]
    x, y = zip(*polygon)
    colors = ['blue'] * (len(x) - 1) + ['red']  # Último ponto em vermelho
    data_dict = {
        "x": x,
        "y": y,
        "mode": "lines+markers",
        "marker": {"color": colors, "size": [10] * (len(x) - 1) + [15] },
        "name": "Polygon",
        "line": {"shape": "linear"}
    }
    frame["data"].append(data_dict)
    fig_dict["frames"].append(frame)

    slider_step = {
        "args": [
            [year],
            {"frame": {"duration": 300, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 300}}
        ],
        "label": year,
        "method": "animate"
    }
    sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()

In [11]:
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# Configuração do layout
fig_dict["layout"]["width"] = 900 
fig_dict["layout"]["height"] = 700
fig_dict["layout"]["xaxis"] = {"range": [xMin, xMax], "title": "X"}
fig_dict["layout"]["yaxis"] = {"range": [yMin, yMax], "title": "Y"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Stage:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# Configuração inicial do polígono
initial_polygon = polygon_stages2[-1]
x, y = zip(*initial_polygon)

colors = ['blue'] * (len(x))
data_dict = {
    "x": x,
    "y": y,
    "mode": "lines+markers",
    "marker": {"color": colors, "size": 15},
    "name": "Polygon",
    "line": {"shape": "linear"}
}
fig_dict["data"].append(data_dict)

# Criação dos frames para cada estágio do polígono
for i, year in enumerate(stages):
    frame = {"data": [], "name": str(i)}
    mapping_colors = np.array(stages[i])[list(map(lambda x: list(nx.get_node_attributes(colored_Graph, 'pos').values()).index(x), polygon_stages2[-1]))]
    colors = ['blue' if c == None else c for c in mapping_colors]
    data_dict = {
        "x": x,
        "y": y,
        "mode": "lines+markers",
        "marker": {"color": colors, "size": 15},
        "name": "Polygon",
        "line": {"shape": "linear"}
    }
    frame["data"].append(data_dict)
    fig_dict["frames"].append(frame)

    slider_step = {
        "args": [
            [i],
            {"frame": {"duration": 300, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 300}}
        ],
        "label": i,
        "method": "animate"
    }
    sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()